## Read

In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn

from glob import iglob

In [2]:
def process_label(df, path):
    df['label'] = 2
    path = path.rstrip('.csv')
    path = f'{path}.txt'
    with open(path, 'r') as fr:
        lines = fr.readlines()
        for i, line in enumerate(lines[:-1]):
            line = line.rstrip('\n').split(',')
            j, l = int(line[0]), int(line[1])
            k = int(lines[i+1].split(',')[0]) - 1
            if k<j:
                print(k)
            df.loc[j:k, 'label'] = l

        line = lines[-1].rstrip('\n').split(',')
        j, l = int(line[0]), int(line[1])
        df.loc[j:, 'label'] = 1


def process_feature(df):
    df['ln_pts'] = np.log(df['pts']+1)
    df['ln_dyn'] = np.log(df['pts_dyn']+1)
    df['ln_sta'] = np.log(df['pts']-df['pts_dyn']+1)

    df['z_iqr'] = df['z_q3'] - df['z_q1']

    df['range'] = np.sqrt(np.square(df['x_c'])+np.square(df['y_c']))

    df['dx'] = df['x_c'].diff().fillna(0)
    df['dy'] = df['y_c'].diff().fillna(0)
    df['dist'] = np.sqrt(np.square(df['dx'])+np.square(df['dy']))

    df.reset_index(inplace=True)


In [3]:
dfs = []

for path in iglob(r'C:/Users/pontosense/Downloads/Target/*.csv'):
    path = path.replace('\\', '/')
    df = pd.read_csv(path, index_col=0)

    process_label(df, path)
    process_feature(df)

    n = round(df.shape[0] / 120)
    print(path.split('/')[-1], n, '{:.1%}'.format(df['label'].mean()))

    df = df[
        ['ln_pts', 'z_iqr', 'z_std', 'z_c', 'range', 'l_r', 's_r', 'dist', 'label']
    ].astype('float32').values
    for i in range(n):
        dfs.append(df)

ATR_GN_P2-240527-30019-2025-08-31-00-1384880-1388055.csv 24 98.1%
ATR_GN_P2-240527-30019-2025-09-02-00-3097855-3098844.csv 8 98.0%
ATR_GN_P2-240527-30019-2025-09-02-09-3427349-3428726.csv 7 100.0%
ATR_GN_P2-240601-00069-2025-08-27-15-181634-185130.csv 28 77.2%
ATR_GN_P2-240601-00069-2025-08-29-04-713100-716599.csv 29 98.6%
ATR_GN_P4-241029-00016-2025-09-01-07-2584784-2591384.csv 49 97.7%
ATR_GN_P4-250120-00138-2025-09-04-15-372127-373946.csv 14 95.0%
ATR_GN_P4-250120-00138-2025-09-07-15-2969150-2970732.csv 13 99.1%
ATR_GN_P4-250120-00138-2025-09-07-20-3155534-3159779.csv 20 7.1%
ATR_GN_P4-250120-00172-2025-09-01-03-7698018-7703053.csv 18 100.0%
P1-TP5-0528test-170695-173549.csv 17 99.8%
P1-TP5-0528test-199959-202507.csv 12 95.8%
P1-TP5-0528test-203583-205322.csv 13 99.4%


## Dataset & Dataloader

In [4]:
from torch.utils.data import (
    Dataset,
    DataLoader
)

In [5]:
class TrainDataset(Dataset):

    def __init__(self, dfs):
        self.dfs = dfs

    def __len__(self):
        return len(self.dfs)

    def __getitem__(self, idx):
        arr = self.dfs[idx]
        jdx = np.random.choice(arr.shape[0]-79, 1)[0]
        batch = arr[jdx:jdx+80, :-1]
        label = arr[jdx:jdx+80, -1]
        return batch, label


In [6]:
np.random.seed(2025)
torch.manual_seed(2025)

train_ds = TrainDataset(dfs)
train_dl = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=0, drop_last=True)

## Model

In [7]:
from torchinfo import summary

from layers import (
    EMA,
    Transpose
)

In [8]:
hs = 64

model = nn.Sequential(
    nn.Linear(8, hs, bias=False),
    Transpose(),
    nn.ReLU6(),
    EMA(hs, length=80),
    Transpose(),
    nn.ReLU6(),
    nn.Linear(hs, 1, bias=False)
)

summary(model, (2, 80, 8), col_names=['input_size', 'output_size', 'num_params'])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
Sequential                               [2, 80, 8]                [2, 80, 1]                --
├─Linear: 1-1                            [2, 80, 8]                [2, 80, 64]               512
├─Transpose: 1-2                         [2, 80, 64]               [2, 64, 80]               --
├─ReLU6: 1-3                             [2, 64, 80]               [2, 64, 80]               --
├─EMA: 1-4                               [2, 64, 80]               [2, 64, 80]               128
├─Transpose: 1-5                         [2, 64, 80]               [2, 80, 64]               --
├─ReLU6: 1-6                             [2, 80, 64]               [2, 80, 64]               --
├─Linear: 1-7                            [2, 80, 64]               [2, 80, 1]                64
Total params: 704
Trainable params: 704
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.01
For

## Train

In [9]:
from torchmetrics import MetricCollection
from torchmetrics.classification import (
    BinaryAccuracy,
    BinaryAUROC
)

In [10]:
# criterion
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=.1**.5)

metric = MetricCollection({'acc':BinaryAccuracy(), 'auc':BinaryAUROC()})

In [11]:
epochs = 300
step = 0

model.train()

for epoch in range(epochs):

    for batch, label in train_dl:
        logit = model(batch).squeeze(-1)
        loss = loss_fn(logit, label)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        proba = torch.sigmoid(logit)
        metric.update(proba, label)
        step += 1
    scheduler.step()

    if (epoch % 10 == 1) or (epoch==epochs-1):
        res = metric.compute()
        print(f'epoch {epoch+1:>2d} - acc: {res["acc"]:.4f} - auc: {res["auc"]:.4f}')
        metric.reset()
        torch.save(model.state_dict(), f'./ckpt/epoch{epoch}-{res["auc"]:.4f}.pth')


epoch  2 - acc: 0.8713 - auc: 0.6142
epoch 12 - acc: 0.8862 - auc: 0.7269
epoch 22 - acc: 0.8822 - auc: 0.8705
epoch 32 - acc: 0.8844 - auc: 0.8996
epoch 42 - acc: 0.8874 - auc: 0.9022
epoch 52 - acc: 0.8883 - auc: 0.9096
epoch 62 - acc: 0.8948 - auc: 0.9173
epoch 72 - acc: 0.8953 - auc: 0.9194
epoch 82 - acc: 0.9093 - auc: 0.9220
epoch 92 - acc: 0.9199 - auc: 0.9289
epoch 102 - acc: 0.9305 - auc: 0.9361
epoch 112 - acc: 0.9326 - auc: 0.9327
epoch 122 - acc: 0.9319 - auc: 0.9333
epoch 132 - acc: 0.9320 - auc: 0.9186
epoch 142 - acc: 0.9319 - auc: 0.9304
epoch 152 - acc: 0.9333 - auc: 0.9348
epoch 162 - acc: 0.9360 - auc: 0.9331
epoch 172 - acc: 0.9346 - auc: 0.9289
epoch 182 - acc: 0.9294 - auc: 0.9273
epoch 192 - acc: 0.9385 - auc: 0.9409
epoch 202 - acc: 0.9403 - auc: 0.9287
epoch 212 - acc: 0.9334 - auc: 0.9346
epoch 222 - acc: 0.9451 - auc: 0.9375
epoch 232 - acc: 0.9358 - auc: 0.9253
epoch 242 - acc: 0.9354 - auc: 0.9327
epoch 252 - acc: 0.9342 - auc: 0.9301
epoch 262 - acc: 0.935

In [12]:
print(f"Epoch {epoch}, lr = {scheduler.get_last_lr()}")

Epoch 299, lr = [1.0000000000000002e-06]


In [15]:
model

Sequential(
  (0): Linear(in_features=8, out_features=64, bias=False)
  (1): Transpose()
  (2): ReLU6()
  (3): EMA(in_chn=64, length=80)
  (4): Transpose()
  (5): ReLU6()
  (6): Linear(in_features=64, out_features=1, bias=False)
)

In [14]:
model[3].alpha.data.flatten()

tensor([0.1497, 0.5854, 0.2912, 0.5000, 0.8639, 0.2013, 0.4864, 0.5199, 0.6250,
        0.4718, 0.9034, 0.8661, 0.5589, 0.5672, 0.8898, 0.5437, 0.2005, 0.6158,
        0.5408, 0.0230, 0.9072, 0.4784, 0.8509, 0.4257, 0.5043, 0.8145, 0.1796,
        0.5348, 0.8724, 0.5836, 0.5419, 0.8934, 0.5584, 0.9504, 0.6122, 0.9176,
        0.2484, 0.4015, 0.7118, 0.1777, 0.6884, 0.2774, 0.5411, 0.6919, 0.4585,
        0.4447, 0.4474, 0.3540, 0.4767, 0.4748, 0.8927, 0.8342, 0.8278, 0.6744,
        0.9054, 0.6385, 0.6868, 0.5670, 0.3053, 0.5882, 0.8954, 0.7668, 0.2954,
        0.6239])

In [16]:
model[3].x0

Parameter containing:
tensor([[0.5236, 1.0781, 0.3415, 0.5092, 0.1020, 0.4146, 0.7162, 0.4957, 0.4507,
         0.7981, 0.2238, 0.8494, 1.2850, 1.1344, 0.5370, 0.5817, 0.9354, 0.3309,
         0.5652, 0.2499, 0.7896, 1.5405, 0.1193, 0.8971, 1.2342, 0.6317, 0.8746,
         0.0140, 0.2358, 1.0745, 0.7006, 0.5269, 0.4790, 0.7471, 1.0356, 0.9406,
         1.1657, 1.1054, 0.9245, 0.1913, 0.1525, 0.4900, 1.2108, 0.7885, 1.1727,
         0.0103, 1.2253, 0.5801, 0.1386, 0.5358, 0.3456, 0.4349, 0.8747, 1.0210,
         1.0829, 0.6584, 1.1456, 0.8914, 0.1315, 0.1847, 1.1154, 1.1015, 1.3611,
         0.4635]], requires_grad=True)

In [17]:
model[0].weight

Parameter containing:
tensor([[ 8.6219e-02,  8.6647e-01,  4.4120e-01,  4.8139e-01,  1.7514e-01,
          1.0891e-02, -1.2926e-01,  1.3097e-01],
        [-7.8895e-02,  9.4671e-02,  3.1408e-01, -1.6057e-02, -1.5108e-01,
         -7.8588e-02, -2.7550e-01, -9.8469e-02],
        [ 2.7193e-01, -1.1324e-01, -6.7713e-01, -4.9525e-01, -1.3370e-02,
         -2.5066e-01, -4.5309e-01,  2.8976e-01],
        [ 1.1430e-01,  2.0001e-01,  7.0208e-01,  7.2217e-01,  2.5706e-01,
         -8.1588e-02,  4.5865e-01,  2.4405e-02],
        [ 6.6334e-02, -4.1747e-01, -3.7110e-01, -1.5670e+00,  3.6219e-01,
         -4.4633e-01,  6.2172e-02,  2.8601e-02],
        [ 4.2632e-02,  2.0036e-01,  7.3452e-02,  1.4585e-01,  2.2702e-01,
         -3.3384e-01, -3.6467e-02,  4.9955e-02],
        [ 2.8507e-03, -3.5115e-01, -2.0289e-01, -3.0688e-01, -2.8920e-01,
         -2.0242e-01, -3.3103e-01,  1.3393e-01],
        [ 1.1236e-01,  8.9976e-01,  1.0464e+00,  1.3387e-01, -2.4479e-01,
          7.5734e-01,  4.7120e-01,  3.0540e

In [18]:
model[6].weight

Parameter containing:
tensor([[ 0.0986,  0.2171, -0.2197,  0.2186, -0.6705, -0.0386,  0.7038,  0.4469,
          0.1479,  0.4008, -0.0671, -0.6016,  0.1744,  0.1328,  0.5334,  0.3945,
          0.2411,  0.1457,  0.0695, -0.0257,  0.4947,  0.5914, -0.4542,  0.0302,
          0.1464,  0.1731,  0.1242, -0.4306,  0.5270,  0.1888,  0.3692, -0.4161,
          0.3582, -0.1187,  0.1349,  0.6904,  0.1906,  0.0952,  0.1687, -0.0470,
         -0.4459,  0.3469,  0.0778,  0.3535,  0.2238, -0.4459,  0.3884,  0.4083,
         -0.1444,  0.0720, -0.3274, -0.0958,  0.0224,  0.3410, -0.1821,  0.1117,
          0.1700,  0.2106,  0.0649,  0.3319, -0.3149,  0.1729,  0.1013,  0.1694]],
       requires_grad=True)